In [1]:
import numpy as np
from gsmmutils import ModelValidator, MyModel
from cobra.flux_analysis import pfba, flux_variability_analysis as fva
import gsmmutils
gsmmutils.DATA_PATH = "../data"
from cobra.io import to_json, save_json_model
from utils import *
import logging
from mewpy.problems import GKOProblem
from mewpy.optimization import EA
logging.getLogger('cobra').setLevel(logging.CRITICAL)
logging.getLogger('gsmmutils').setLevel(logging.CRITICAL)
logging.getLogger('mewpy').setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [18]:
model = MyModel(r"../data/model_no_carotenoids.xml", "e_Biomass__cytop")
for demand in model.demands:
    if demand.id not in ["DM_C02094__chlo", "DM_C08601__chlo"] and 'photon' not in demand.id:
        demand.bounds = (0, 0)
model.exchanges.EX_C00011__dra.bounds = (-8.61, 1000)
model.reactions.e_Pigment__chlo.bounds = (0,0)
constraint = model.problem.Constraint(
            model.reactions.R09503_hn438__lum.flux_expression + model.reactions.R09503_hn673__lum.flux_expression,
            lb=0,
            ub=199.44)
model.add_cons_vars(constraint)
model.summary()

Loading

Reactions: 4623
Metabolites: 3737
Genes: 1690
Model loaded
Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,7.08,0,0.00%
C00009__extr,EX_C00009__dra,0.0332,0,0.00%
C00011__extr,EX_C00011__dra,8.61,1,99.75%
C00059__extr,EX_C00059__dra,0.02207,0,0.00%
C00080__extr,EX_C00080__dra,0.05599,0,0.00%
C00120__extr,EX_C00120__dra,0.0008005,10,0.09%
C00205__extr,EX_C00205__dra,618.7,0,0.00%
C00244__extr,EX_C00244__dra,0.5456,0,0.00%
C00305__extr,EX_C00305__dra,0.0001543,0,0.00%
C00378__extr,EX_C00378__dra,0.0004601,12,0.06%


In [19]:
lutein_range = [e/1000000 for e in range(100, int(0.00010*100*1000000), 100)]

In [20]:
for lutein_content in lutein_range:
    model, _ = adjust_biomass(model, model.metabolites.C08601__chlo, model.reactions.e_Pigment__chlo, model.metabolites.e_Pigment__chlo, lutein_content)
    model, _ = adjust_biomass(model, model.metabolites.C02094__chlo, model.reactions.e_Pigment__chlo, model.metabolites.e_Pigment__chlo, lutein_content)

In [21]:
[setattr(x, 'objective_coefficient', 0) for x in model.reactions if x.objective_coefficient != 0]
[setattr(x, 'objective_coefficient', 1) for x in model.reactions if "Biomass" in x.id and "_v" in x.id]
print()

In [22]:
model.summary()

Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,7.084,0,0.00%
C00009__extr,EX_C00009__dra,0.03333,0,0.00%
C00011__extr,EX_C00011__dra,8.61,1,99.75%
C00059__extr,EX_C00059__dra,0.02216,0,0.00%
C00080__extr,EX_C00080__dra,0.05622,0,0.00%
C00120__extr,EX_C00120__dra,0.0008038,10,0.09%
C00205__extr,EX_C00205__dra,618.4,0,0.00%
C00244__extr,EX_C00244__dra,0.5478,0,0.00%
C00305__extr,EX_C00305__dra,0.000133,0,0.00%
C00378__extr,EX_C00378__dra,0.000462,12,0.06%


In [45]:
po4_range = np.arange(0.01, 0.035, 0.005)
nitrate_range = np.arange(0.2, 1, 0.05)
model.reactions.e_Biomass__cytop.bounds = (0, 0)
for nitrate in nitrate_range:
    for po4 in po4_range:
        with model:
            print(nitrate, po4)
            model.exchanges.EX_C00009__dra.bounds = (-po4, -po4)
            model.exchanges.EX_C00244__dra.bounds = (-nitrate, -nitrate)
            try:
                slim = model.slim_optimize()
                sol_pfba = model.maximize(pfba=True, value=False)
                if slim > 0 and sol_pfba and sol_pfba.status == 'optimal':
                    sol_fva = fva(model, [reaction.id for reaction in model.reactions if "Biomass" in reaction.id and "_v" in reaction.id])
                    sol_pfba_df = (sol_pfba.fluxes.loc[sol_pfba.fluxes.index.str.contains("e_Biomass_v")])    
                    merged = pd.concat([sol_pfba_df, sol_fva], axis=1)
                    print(merged.loc[round(merged.maximum, 4) > 0])
                print("#"*100)
            except:
                pass

0.2 0.01
                         fluxes   minimum   maximum
e_Biomass_v198__cytop  0.064315  0.064315  0.064315
####################################################################################################
0.2 0.015
None (infeasible).
####################################################################################################
0.2 0.019999999999999997
None (infeasible).
####################################################################################################
0.2 0.024999999999999998
None (infeasible).
####################################################################################################
0.2 0.03
None (infeasible).
####################################################################################################
0.25 0.01
                         fluxes   minimum   maximum
e_Biomass_v198__cytop  0.064315  0.064315  0.064315
####################################################################################################
0.25 0.015
           

In [43]:
model.reactions.EX_C00009__dra.bounds = (-0.033, -0.033)
sol_fva = fva(model, [reaction.id for reaction in model.reactions if "Biomass" in reaction.id and "_v" in reaction.id] + 
             ["EX_C00244__dra"])
sol_fva.loc[sol_fva.maximum != 0]

,minimum,maximum
e_Biomass_v1__cytop,0.0,0.160307
e_Biomass_v2__cytop,0.0,0.146132
e_Biomass_v3__cytop,0.0,0.146881
e_Biomass_v4__cytop,0.0,0.147639
e_Biomass_v5__cytop,0.0,0.148404
...,...,...
e_Biomass_v195__cytop,0.0,0.064123
e_Biomass_v196__cytop,0.0,0.063792
e_Biomass_v197__cytop,0.0,0.063465
e_Biomass_v198__cytop,0.0,0.063141


In [25]:
po4_range

array([0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 , 0.045])

In [ ]:
model.reactions.e_Biomass_v198__cytop.backtrace

In [ ]:
model.objective.expression